<a href="https://colab.research.google.com/github/hemantsinghrajput/Text_classification/blob/main/Text_classification_using_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install transformers[torch] pandas datasets pyarrow scikit-learn
!pip install simpletransformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!nvidia-smi

Fri Mar 10 07:21:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [98]:
import pandas as pd
df1 = pd.read_csv('/content/test.csv')
df2 = pd.read_csv('/content/train.csv')

df2


,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [99]:
df1

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
...,...,...,...
7595,1,Around the world,Ukrainian presidential candidate Viktor Yushch...
7596,2,Void is filled with Clement,With the supply of attractive pitching options...
7597,2,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...
7598,3,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...


In [100]:
df=pd.concat([df1, df2])

In [101]:
df['Title']=df['Title']+" "+df['Description']

In [102]:
df=df.dropna()

In [103]:
df['Class Index'] =df['Class Index'].replace(1,0)
df['Class Index'] =df['Class Index'].replace(2,1)
df['Class Index'] =df['Class Index'].replace(3,2)
df['Class Index'] =df['Class Index'].replace(4,3)

In [104]:
df

,Class Index,Title,Description
0,2,Fears for T N pension after talks Unions repre...,Unions representing workers at Turner Newall...
1,3,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,3,Ky. Company Wins Grant to Study Peptides (AP) ...,AP - A company founded by a chemistry research...
3,3,Prediction Unit Helps Forecast Wildfires (AP) ...,AP - It's barely dawn when Mike Fitzpatrick st...
4,3,Calif. Aims to Limit Farm-Related Smog (AP) AP...,AP - Southern California's smog-fighting agenc...
...,...,...,...
119995,0,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,1,Renteria signing a top-shelf deal Red Sox gene...,Red Sox general manager Theo Epstein acknowled...
119997,1,Saban not going to Dolphins yet The Miami Dolp...,The Miami Dolphins will put their courtship of...
119998,1,Today's NFL games PITTSBURGH at NY GIANTS Time...,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [105]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
     

In [106]:
df["Class Index"].value_counts()


2    31900
3    31900
1    31900
0    31900
Name: Class Index, dtype: int64

In [107]:
TEXT_LABELS = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

In [108]:
#Independent and dependent variable
x = df["Title"]
y = df["Class Index"]
     

In [109]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)


In [21]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs


In [27]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.learning_rate = 1e-5

In [28]:
model = ClassificationModel("bert", "bert-base-cased",num_labels = 4,args=model_args,use_cuda=True)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [29]:
train_df = pd.DataFrame({"Title":x_train, "Class Index":y_train})
eval_df = pd.DataFrame({"Title":x_test, "Class Index":y_test})

In [30]:

train_df.head()


,Title,Class Index
27402,IBM Expands RFID Offerings (NewsFactor) NewsFa...,3
1590,"Stocks Up on Earnings, Oil, Economic Data NEW...",2
935,Google: Now Playboy's Latest Bunny (washington...,3
64493,US to appeal against WTO ruling The United Sta...,2
80550,Time Warner 3rd-Qtr Profit Falls 7.8 on Legal ...,2


In [31]:
model.train_model(train_df, acc=accuracy_score)


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/102080 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/12760 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/12760 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/12760 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/12760 [00:00<?, ?it/s]

(51040, 0.1782265858526187)

In [118]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=accuracy_score)


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/25520 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3190 [00:00<?, ?it/s]

In [51]:
from joblib import Parallel, delayed
import joblib


# Save the model as a pickle in a file
joblib.dump(model, '/content/model5.pkl')

# Load the model from the file
knn_from_joblib = joblib.load('/content/model5.pkl')

# Use the loaded model to make predictions
#knn_from_joblib.predict(X_test)
knn_from_joblib

In [119]:
result

{'mcc': 0.9286943237372353,
 'acc': 0.9465125391849529,
 'eval_loss': 0.29872286713982826}

In [120]:
x_test.iloc[4]

'NHL Must Determine if It Is Certain About Cost Certainty After Gary Bettman was introduced as the commissioner of the National Hockey League 12 years ago, he was handed a fax from Bob Goodenow, the executive director of the players association.'

In [121]:
x_test.values

array(['MPAA to sue Net pirates Warning that film piracy represents the greatest economic threat to the movie industry in its 100 year-plus history, the Motion Picture Association of America announced Thursday that it plans ',
       'IBM, LG winding down joint venture Sources say the two companies will likely dissolve their 8-year-old subsidiary sometime this year.',
       'Health workers fear deadly epidemic in Haiti Bodies, dirty water spreading disease. Medical supplies have started to run out and panicked residents are fighting for provisions.',
       ...,
       'Falconbridge Profit Rises Eightfold on Nickel Prices (Update3) Falconbridge Ltd., the world #39;s third- biggest nickel producer, said third-quarter earnings rose more than eightfold as base-metal prices jumped.',
       'Yankees Could Be Without Rivera Early in ALCS (Reuters) Reuters - New York Yankees closer Mariano\\Rivera is reportedly dealing with a personal issue that could\\affect his availability early in the A

In [122]:
preds, model_ouputs = model.predict(list(x_test.values))


  0%|          | 0/25520 [00:00<?, ?it/s]

  0%|          | 0/3190 [00:00<?, ?it/s]

In [123]:
preds

array([3, 3, 0, ..., 2, 1, 1])

In [124]:
model_ouputs

array([[-2.94921875, -3.875     , -1.23242188,  7.56640625],
       [-2.82421875, -4.03125   , -1.15136719,  7.3203125 ],
       [ 7.19921875, -4.1796875 , -2.4375    , -1.79785156],
       ...,
       [-1.48242188, -3.21484375,  7.2421875 , -2.21679688],
       [-0.89941406,  8.6875    , -3.48046875, -3.20703125],
       [-2.61132812,  9.203125  , -2.53515625, -2.76953125]])

In [125]:
out_put=model.predict(["Google: Now Playboy's Latest Bunny (washington"])[0][0]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [126]:
for x in TEXT_LABELS:
  if(x==out_put):
    print(TEXT_LABELS[x])

Sci/Tech


In [127]:
out_put=model.predict(["Around the world"])[0][0]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [128]:
for x in TEXT_LABELS:
  if(x==out_put):
    print(TEXT_LABELS[x])

World


In [129]:
y_test=y_test.reset_index(drop=True)

In [130]:
y_test

0        3
1        3
2        0
3        3
4        1
        ..
25515    1
25516    3
25517    2
25518    1
25519    1
Name: Class Index, Length: 25520, dtype: int64

In [131]:
from sklearn.metrics import precision_score,recall_score
print("Precision Score : ",precision_score(y_test, preds, 
                                           pos_label='positive',
                                           average='micro'))
print("Recall Score : ",recall_score(y_test, preds, 
                                           pos_label='positive',
                                           average='micro'))

Precision Score :  0.9811128526645768
Recall Score :  0.9811128526645768


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [132]:
accuracy = accuracy_score(y_test, preds)
print('Accuracy:', accuracy)
f1 = f1_score(y_test, preds,pos_label='positive',
                                           average='micro')
print('F1-score:', f1)

Accuracy: 0.9811128526645768
F1-score: 0.9811128526645768


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
